In [2]:
#regex for special characters
from random import random
#from modules.tweet_collector import *
import os, re, pandas as pd
from datetime import datetime

In [3]:

main_path = os.getcwd()

politifact = pd.read_csv(main_path+"/data/fakenews_sources/politifact_scape_2609.csv")

dont_sample = politifact[politifact.claim.str.contains('don\'t')]

sample = dont_sample.sample(10, random_state=42).reset_index()

sample.claim[4]

'CDC "will revoke the emergency use authorization" of the PCR COVID tests because those tests don\'t differentiate between COVID and flu viruses. "Translation: They\'ve been adding flu cases to Covid cases when using that test.”'

In [4]:
#Cleaning step 1 - getting rid of \n appearances and weird spaces
sample['claim'] = sample.claim.apply(lambda x: x.strip())

#Cleaning from all special characters - perhaps find trick like \w\'\w is to avoid removing the \' in don't, won't they've etc.
sample['claim'] = sample.claim.apply(lambda x: re.sub(r'[^a-zA-Z0-9 ]', '', x))

#Removing "Says" in the beginning of a statement around a thoussand in the sample have this structure.
sample['claim'] = sample.claim.apply(lambda x: re.sub(r'^Says ', '', x))


#Resources and ideas
#https://datagy.io/python-remove-special-characters-from-string/

# #alternative approach
# sample['clean_claim'] = sample.claim.apply(lambda x: [re.sub(r"[^a-zA-Z0-9]+", ' ', k) for k in x.split("\n")][0].strip())

In [5]:
sample.claim[4]

'CDC will revoke the emergency use authorization of the PCR COVID tests because those tests dont differentiate between COVID and flu viruses Translation Theyve been adding flu cases to Covid cases when using that test'

In [70]:
date = 'November 25, 2018'
datet = str(datetime.strptime(date, '%B %d, %Y').isoformat()+"Z")



In [71]:
datet

'2018-11-25T00:00:00Z'

In [53]:
date_str = '09-19-2018'

date_object = datetime.strptime(date_str, '%m-%d-%Y')[0:-4].date()


<class 'datetime.date'>


In [74]:
str(datetime.today().isoformat()[0:-4]+"Z")

'2022-09-29T11:19:05.86Z'